# Projeto 2 - Como pegar dados de um site com Python? - Pegando dados de ETFs do mundo inteiro.


### Desafio:

* Construir um código que vá no site etf.com e busque dados de todos os etfs do mercado americano e, consequentemente, do mundo. Rentabilidades, patrimônio, gestora, taxa...   
* Lembrar de sempre trazer outros tipos de cenários onde a pessoa precisa pegar dados de sites etc.


### Passo a passo:

   **Passo 1** - Definir um navegador que você irá utilizar para navegar com o Python.

   **Passo 2** - Importar os módulos e bibliotecas.
   
   **Passo 3** - Entender como funcionam requisições na internet.
   
   **Passo 4** - Conhecer e mapear o processo de coleta de dados no site do ETF.com. 
   
   **Passo 5** - Achar todos os elementos necessários dentro do HTML do site.
   
   **Passo 6** - Ler a tabela de dados.
   
   **Passo 7** - Construir a tabela final.

# Passo 1: Escolher o navegador.

No nosso caso, utilizaremos o Google Chrome. 

# Passo 2: Importar as bibliotecas.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Passo 3: Entender como funcionam requisições na internet.

In [ ]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

driver.get('https://www.etf.com/etfanalytics/etf-finder')

# Passo 4: Conhecer e mapear o processo de coleta de dados no site do ETF.com.

# Passo 5.1: Achar todos os elementos necessários dentro do HTML do site - Expandindo a tabela para 100 itens.

In [ ]:
time.sleep(5)

botao_100 = driver.find_element("xpath",
                               '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span')

botao_100.click()

#driver.execute_script("arguments[0].click();", botao_100)


# Passo 5.2: Achar todos os elementos necessários dentro do HTML do site - Pegando o número de páginas da tabela.

In [ ]:
numero_paginas = driver.find_element("xpath", '//*[@id="totalPages"]')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

numero_paginas

# Passo 6.1: Lendo a tabela de dados - Lendo a tabela de dados básicos.

In [ ]:
elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

html_tabela = elemento.get_attribute('outerHTML')

tabela = pd.read_html(str(html_tabela))[0]

tabela

In [ ]:
lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    botao_avancar_pagina.click()
    
    
tabela_cadastro_etfs = pd.concat(lista_tabela_por_pagina)

tabela_cadastro_etfs

# Passo 6.2: Ler a tabela de dados - Preenchendo um campo no site pra voltar as páginas

In [ ]:
formulario_de_voltar_pagina = driver.find_element("xpath", '//*[@id="goToPage"]')

formulario_de_voltar_pagina.clear()
formulario_de_voltar_pagina.send_keys("1")
formulario_de_voltar_pagina.send_keys(u'\ue007')

# Passo 6.3: Ler a tabela de dados - Lendo a tabela de dados de rentabilidade.

In [ ]:
botao_mudar_pra_performance = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')

botao_mudar_pra_performance.click()

# a partir daqui é tudo igual

lista_tabela_por_pagina = []

elemento = driver.find_element("xpath", '//*[@id="finderTable"]')

for pagina in range(1, numero_paginas + 1):
    
    html_tabela = elemento.get_attribute('outerHTML')
    
    tabela = pd.read_html(str(html_tabela))[0]
    
    lista_tabela_por_pagina.append(tabela)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    
    
tabela_rentabilidade_etfs = pd.concat(lista_tabela_por_pagina)

tabela_rentabilidade_etfs

In [ ]:
driver.quit()

In [ ]:
tabela_cadastro_etfs

In [ ]:
tabela_rentabilidade_etfs

# Passo 7: Construindo a tabela final.

In [ ]:
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs.set_index("Ticker")
tabela_rentabilidade_etfs = tabela_rentabilidade_etfs[['1 Year', '3 Years', '5 Years']]
tabela_cadastro_etfs = tabela_cadastro_etfs.set_index("Ticker")

In [ ]:
base_de_dados_final = tabela_cadastro_etfs.join(tabela_rentabilidade_etfs, how = 'inner')

base_de_dados_final